# 3.1 Transforming CSV data to RDF

Manually creating RDF graphs, as was done in the previous parts of this tutorial, is slow. This needs to be automated further. In this part of the tutorial, we will create RDF graphs from CSV files (comma separated values). We will do this for a simple building model, and rely on the ontology that we built earlier.

## 1. The sample building

For this part, we will assume a building that has the following layout, and that relies on the ontology that we created earlier and that allows us to model spaces, sensors, walls, and a few more things. The following building we will represent in an Excel file and CSV file (exported from Excel), such that the RDF graph can be created automatically.

<img src="figures/simpleBuildingLayout.png" width="600" />

Responding to our ontology, our building structure is represented in Excel as follows (see file [simplebuilding.xlsx](data/simplebuilding.xlsx) and [simplebuilding.csv](data/simplebuilding.csv)):

<img src="figures/simpleBuildingExcel.png" width="400" />

## 2. Parsing the CSV data
We first load in the CSV file, as well as the ontology graph, using the fitting packages, namely `rdflib`, and `csv` or `pandas`.

In [45]:
pip install pandas

In [61]:
from rdflib import Graph, OWL, RDF, RDFS, Namespace, URIRef, Literal

#instance graph and ontology
g = Graph()
ontology_graph = Graph()
ontology_graph.parse("output/myFirstOntology.ttl")

import csv
with open('data/simplebuilding.csv', 'r') as csv_file:
    reader = csv.reader(csv_file)
    for row in reader:
        print(row)

import pandas
df = pandas.read_csv('data/simplebuilding.csv')
print(df)

['ï»¿Space', 'Element', 'Element Type']
['Kitchen', 'Wall 1', 'Wall']
['Kitchen', 'Wall 2', 'Wall']
['Kitchen', 'Wall 5', 'Wall']
['Kitchen', 'Wall 6', 'Wall']
['Kitchen', 'Sensor 1', 'Sensor']
['Bedroom', 'Wall 1', 'Wall']
['Bedroom', 'Wall 2', 'Wall']
['Bedroom', 'Wall 7', 'Wall']
['Bedroom', 'Wall 3', 'Wall']
['Living Space', 'Wall 5', 'Wall']
['Living Space', 'Wall 4', 'Wall']
['Living Space', 'Wall 6', 'Wall']
['Living Space', 'Wall 7', 'Wall']
['Living Space', 'Wall 3', 'Wall']
['Living Space', 'AHU 1', 'AirHandlingUnit']
           Space   Element     Element Type
0        Kitchen    Wall 1             Wall
1        Kitchen    Wall 2             Wall
2        Kitchen    Wall 5             Wall
3        Kitchen    Wall 6             Wall
4        Kitchen  Sensor 1           Sensor
5        Bedroom    Wall 1             Wall
6        Bedroom    Wall 2             Wall
7        Bedroom    Wall 7             Wall
8        Bedroom    Wall 3             Wall
9   Living Space    Wall 5

## 3. Setting up the RDFLib basics
Next, we load some basic settings, similar to previous parts of this tutorial.

In [47]:
g.bind("owl", OWL)
MFI = Namespace("https://example.org/myFirstOntology#")
g.bind("mfi", MFI)

# Add namespace and prefix for instance graph (ABox)
INST = Namespace("https://example.org/myFirstInstanceGraph/")
g.bind("", INST) # bind to default empty prefix
g.bind("inst", INST) # bind to inst prefix

# Initiate ontology entity
s = URIRef("https://example.org/myFirstInstanceGraph")
p = RDF.type
o = OWL.Ontology
g.add((s, p, o))

<Graph identifier=Ndcb528b67f5249caa979a2fa001ecab1 (<class 'rdflib.graph.Graph'>)>

## 4. Instantiating spaces in the graph

Next, we will retrieve all unique Spaces and Elements in this CSV spreadsheet, and instantiate those. These are our unique spaces, elements, and elementtypes:

In [63]:
uniqueSpaces = df['Space'].unique()
uniqueElements = df['Element'].unique()
uniqueElementTypes = df['Element Type'].unique()

Then we create all the spaces that we have in column one of our Excel table input.

In [49]:
# first create all the spaces, that is easy
i = 1
for space in uniqueSpaces:
    sp = URIRef("https://example.org/myFirstInstanceGraph/" + "space_" + str(i))
    g.add((sp, RDF.type, MFI["Space"]))
    g.add((sp, MFI["name"], Literal(space)))

    # check the created value
    print(f"We have now a space with name {g.value(sp, MFI.name)}")

    #add +1 to index counter
    i+=1

We have now a space with name Kitchen
We have now a space with name Living Space
We have now a space with name Bedroom


## 5. Instantiating the building elements in the graph
To add the correct elements to each space, we will need to iterate over each row of the input, to be able to match the correct elements with the correct spaces. So that means that we will anyway need to iterate over the rows of our Excel input (`DataFrame`). Since we will iterate over these values, we can then also create these elements during that iteration. We can iterate over our input as follows:

In [50]:
# Iterate over the rows using iterrows() to add elements to spaces
for index, row in df.iterrows():
    # get values
    spaceval = row['Space']
    elementval = row['Element']
    elementtypeval = row['Element Type']

At every row, we need to select the corresponding `Space` instance from the `Graph`, and make sure also that it exists. This can be done by a *Contains check* (see also https://rdflib.readthedocs.io/en/stable/intro_to_graphs.html#contains-check).

In [51]:
exampleSpace = URIRef("https://example.org/myFirstInstanceGraph/space_1")
if (exampleSpace, RDF.type, MFI.Space) in g:
    print("This graph knows that Space_1 is a mfi:Space!")

if (exampleSpace, None, None) in g:
    print("This graph contains triples about Space_1!")

This graph knows that Space_1 is a mfi:Space!
This graph contains triples about Space_1!


However, since we don't know the correct URI, this method is not the best one to use. We will instead query for spaces that have the same label as the one found in the Excel spreadsheet. This is a potential vulnerability, since matching triples based on string labels is generally unreliable (two spaces could have the same name label).

In [52]:
i = 1
# Iterate over the rows using iterrows() to add elements to spaces
for index, row in df.iterrows():
    # get values
    spaceval = row['Space']
    elementval = row['Element']
    elementtypeval = row['Element Type']

    # Check if space already exists
    ourQuery = f"""PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE {{ 
            ?space rdf:type mfi:Space .
            ?space mfi:name "{spaceval}"
          }}"""

    print(ourQuery)

    qres = g.query(ourQuery)
    print("length = " + str(len(list(qres))))
    print()

PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE { 
            ?space rdf:type mfi:Space .
            ?space mfi:name "Kitchen"
          }
length = 1

PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE { 
            ?space rdf:type mfi:Space .
            ?space mfi:name "Kitchen"
          }
length = 1

PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE { 
            ?space rdf:type mfi:Space .
            ?space mfi:name "Kitchen"
          }
length = 1

PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE { 
            ?space rdf:type mfi:Space .
            ?space mfi:nam

If the queried space exists, then the output length will be 1, otherwise, it will be zero. The latter option should not be a possibility, since we created all spaces beforehand already. But it can be checked nonetheless. This is included below as an if-else switch. The below will result in a `spaceURIRef` variable that holds the space URI for each row.

In [53]:
i = 1
# Iterate over the rows using iterrows() to add elements to spaces
for index, row in df.iterrows():
    # get values
    spaceval = row['Space']
    elementval = row['Element']
    elementtypeval = row['Element Type']

    # create an empty placeholder to contain the correct spaceURIRef for each row
    spaceURIRef = "placeholder"

    # Check if space already exists
    ourQuery = f"""PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE {{ 
            ?space rdf:type mfi:Space .
            ?space mfi:name "{spaceval}"
          }}"""

    #print(ourQuery)

    qres = g.query(ourQuery)
    #print("length = " + str(len(list(qres))))

    # If the space does not exist, create a new one
    if len(list(qres)) == 0:
        # This is actually not possible here, since we created these spaces already all before
        g.add((INST["space_"+str(i)], RDF.type, MFI["Space"]))    
        g.add((INST["space_"+str(i)], MFI["name"], Literal(space)))
        i += 1
    
    # If yes, fetch the existing one
    else:
        # normally only one element in list, IF their names are unique
        for row in qres:
            spaceURIRef = row.space
            print("found this space: " + str(spaceURIRef))

found this space: https://example.org/myFirstInstanceGraph/space_1
found this space: https://example.org/myFirstInstanceGraph/space_1
found this space: https://example.org/myFirstInstanceGraph/space_1
found this space: https://example.org/myFirstInstanceGraph/space_1
found this space: https://example.org/myFirstInstanceGraph/space_1
found this space: https://example.org/myFirstInstanceGraph/space_2
found this space: https://example.org/myFirstInstanceGraph/space_2
found this space: https://example.org/myFirstInstanceGraph/space_2
found this space: https://example.org/myFirstInstanceGraph/space_2
found this space: https://example.org/myFirstInstanceGraph/space_2
found this space: https://example.org/myFirstInstanceGraph/space_2
found this space: https://example.org/myFirstInstanceGraph/space_3
found this space: https://example.org/myFirstInstanceGraph/space_3
found this space: https://example.org/myFirstInstanceGraph/space_3
found this space: https://example.org/myFirstInstanceGraph/spa

Great, so now that we have the correct `Space` URI for each row in the CSV input, let's look at the elements that are in the CSV file, in column 2 and 3. These have not been created beforehand, so we need to create them as we go. Luckily, if a URI already exists in a graph, `rdflib` does not create a duplicate. So we can simply generate these elements directly, and place them in the correct space.

In [56]:
i = 1
# Iterate over the rows using iterrows() to add elements to spaces
for index, row in df.iterrows():
    elementval = row['Element']
    elementtypeval = row['Element Type']
        
    elementURI = URIRef("https://example.org/myFirstInstanceGraph/" + elementval.replace(" ", "_"))

    g.add((elementURI, RDF.type, MFI["Element"]))    
    g.add((elementURI, MFI["name"], Literal(elementval)))

    print("created element : " + str(elementURI) + " with name " + str(elementval))

created element : https://example.org/myFirstInstanceGraph/Wall_1 with name Wall 1
created element : https://example.org/myFirstInstanceGraph/Wall_2 with name Wall 2
created element : https://example.org/myFirstInstanceGraph/Wall_5 with name Wall 5
created element : https://example.org/myFirstInstanceGraph/Wall_6 with name Wall 6
created element : https://example.org/myFirstInstanceGraph/Sensor_1 with name Sensor 1
created element : https://example.org/myFirstInstanceGraph/Wall_5 with name Wall 5
created element : https://example.org/myFirstInstanceGraph/Wall_4 with name Wall 4
created element : https://example.org/myFirstInstanceGraph/Wall_6 with name Wall 6
created element : https://example.org/myFirstInstanceGraph/Wall_7 with name Wall 7
created element : https://example.org/myFirstInstanceGraph/Wall_3 with name Wall 3
created element : https://example.org/myFirstInstanceGraph/AHU_1 with name AHU 1
created element : https://example.org/myFirstInstanceGraph/Wall_1 with name Wall 1
cr

## 6. Element Types to be added to the graph
Of course, these instances should not just be of type `Element`, but rather one of the `ElementType` classes listed in column three of the input CSV file (and to be present in the `mfi` ontology). So let's correct that. While doing this, we can also check whether these Element Types are present in the ontology, and skip those element types that are not available in the ontology. 

In [58]:
i = 1
# Iterate over the rows using iterrows() to add elements to spaces
for index, row in df.iterrows():
    elementtypeval = row['Element Type']
    
    # get the element type from the ontology
    if (MFI[elementtypeval], RDF.type, OWL.Class) in ontology_graph:
        print(f"Element Type {str(elementtypeval)} is in the MFI ontology.")
    else:
        print(f"Element Type {str(elementtypeval)} is NOT in the MFI ontology.")


Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Sensor is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type AirHandlingUnit is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.
Element Type Wall is in the MFI ontology.


## 7. Summary for creation of the graph
Now we can combine some of the above scripts, and add the element with the correct class from our `mfi` ontology. We can in principle also add it immediately to the correct space, and that leads to the following script.

In [64]:
# Empty our graph
g.remove((None, None, None))

# setup the initial namespaces and prefixes again
g.bind("owl", OWL)
MFI = Namespace("https://example.org/myFirstOntology#")
g.bind("mfi", MFI)

# Add namespace and prefix for instance graph (ABox)
INST = Namespace("https://example.org/myFirstInstanceGraph/")
g.bind("", INST) # bind to default empty prefix
g.bind("inst", INST) # bind to inst prefix

# Initiate ontology entity
s = URIRef("https://example.org/myFirstInstanceGraph")
p = RDF.type
o = OWL.Ontology
g.add((s, p, o))

# first create all the spaces, that is easy
i = 1
for space in uniqueSpaces:
    sp = URIRef("https://example.org/myFirstInstanceGraph/" + "space_" + str(i))
    g.add((sp, RDF.type, MFI["Space"]))
    g.add((sp, MFI["name"], Literal(space)))

    # check the created value
    print(f"We have now a space with name {g.value(sp, MFI.name)}")

    #add +1 to index counter
    i+=1

i = 1
# Iterate over the rows using iterrows() to add elements to spaces
for index, row in df.iterrows():
    # get values
    spaceval = row['Space']
    elementval = row['Element']
    elementtypeval = row['Element Type']

    # create an empty placeholder to contain the correct spaceURIRef for each row
    spaceURIRef = "placeholder"

    # Check if space already exists
    ourQuery = f"""PREFIX mfi: <https://example.org/myFirstOntology#>
        PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

        SELECT ?space
        WHERE {{ 
            ?space rdf:type mfi:Space .
            ?space mfi:name "{spaceval}"
          }}"""

    qres = g.query(ourQuery)

    # If the space does not exist, create a new one
    if len(list(qres)) == 0:
        # This is actually not possible here, since we created these spaces already all before
        g.add((INST["space_"+str(i)], RDF.type, MFI["Space"]))    
        g.add((INST["space_"+str(i)], MFI["name"], Literal(space)))
        i += 1
    
    # If yes, fetch the existing one
    else:
        # normally only one element in list, IF their names are unique
        for row in qres:
            spaceURIRef = row.space
            print("found this space: " + str(spaceURIRef))

    # Create elements based on the values in column 2 of the input CSV file        
    elementURI = URIRef("https://example.org/myFirstInstanceGraph/" + elementval.replace(" ", "_"))

    # check the element type from the ontology and skip the row if it does not exist.
    if (MFI[elementtypeval], RDF.type, OWL.Class) in ontology_graph:
        print(f"Element Type {str(elementtypeval)} is in the MFI ontology.")
    else:
        print(f"Element Type {str(elementtypeval)} is NOT in the MFI ontology.")
        next

    # add the element with the correct element type
    g.add((elementURI, RDF.type, MFI[elementtypeval]))    
    g.add((elementURI, MFI["name"], Literal(elementval)))
    print("created element : " + str(elementURI) + " with name " + str(elementval))

    # add the element to the corresponding space
    g.add((elementURI, MFI.hasLocation, spaceURIRef))


# write out everything to an RDF file.
import os
g.serialize(destination="output/ourCSVbuilding.ttl", format="turtle")
print("Created output/ourCSVbuilding.ttl in folder:")
print(str(os.getcwd()))

We have now a space with name Kitchen
We have now a space with name Bedroom
We have now a space with name Living Space
found this space: https://example.org/myFirstInstanceGraph/space_1
Element Type Wall is in the MFI ontology.
created element : https://example.org/myFirstInstanceGraph/Wall_1 with name Wall 1
found this space: https://example.org/myFirstInstanceGraph/space_1
Element Type Wall is in the MFI ontology.
created element : https://example.org/myFirstInstanceGraph/Wall_2 with name Wall 2
found this space: https://example.org/myFirstInstanceGraph/space_1
Element Type Wall is in the MFI ontology.
created element : https://example.org/myFirstInstanceGraph/Wall_5 with name Wall 5
found this space: https://example.org/myFirstInstanceGraph/space_1
Element Type Wall is in the MFI ontology.
created element : https://example.org/myFirstInstanceGraph/Wall_6 with name Wall 6
found this space: https://example.org/myFirstInstanceGraph/space_1
Element Type Sensor is in the MFI ontology.
cr

## 8. Storing to file and database
Now inspect the resulting TTL file, in Notepad or OntoText GraphDB, and verify that it contains what we had originally specified:

<img src="figures/simpleBuildingLayout.png" width="600" />

One final step that we can take for this dataset, is to load it directly into a graph database or triple store. While serialization to files is nice, we will need to use graph databases to properly scale up our work. So in the below steps, we will connect to a local OntoText GraphDB installation, and load our data into a repository of that installation.

Installation and setup of this OntoText GraphDB software is not discussed here. Please follow the instructions that are available here: https://www.ontotext.com/products/graphdb/.
- Install OntoText GraphDB
- Make new repository with a name like `SSoLDAC2024`
- Try the online interface https://localhost:7200/ to explore your database installation and know what you are working with.

After taking the above steps, the online interface should show the below content when going to https://localhost:7200/, and after creating the correct repository.

<img src="figures/graphDBInterface.png" width="900" />



To communicate with this database, the `requests` package can be used, and needs to be installed first.

In [66]:
pip install requests

   ---------------------------------------- 0.0/64.9 kB ? eta -:--:--
   ---------------------------------------- 64.9/64.9 kB 1.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/164.4 kB ? eta -:--:--
   --------------------------------------- 164.4/164.4 kB 10.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/100.4 kB ? eta -:--:--
   ---------------------------------------- 100.4/100.4 kB ? eta 0:00:00
   ---------------------------------------- 0.0/66.8 kB ? eta -:--:--
   ---------------------------------------- 66.8/66.8 kB 3.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/121.1 kB ? eta -:--:--
   ---------------------------------------- 121.1/121.1 kB 6.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


The below example shows how one can send a simple SELECT query to one of the repositories of this Graph Database. The below query selects all `bot:Building` instances in the database.

In [70]:
import requests

# setup the endpoint that you want to query
endpoint = "https://localhost:7200/repositories/SSoLDAC2024" # change this name 
print( "Will connect to endpoint: " , endpoint )

headers = { "Accept" : "application/sparql-results+json" ,
            "Content-Type" : "application/x-www-form-urlencoded" }

# Setup the query
query = """PREFIX bot: <https://w3id.org/bot#>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>

SELECT ?building
WHERE{ ?building rdf:type bot:Building }"""

# send the query to the endpoint using the requests package
req = requests.post( endpoint ,
                     headers = headers ,
                     data = "query=" + query )

# Response status code, should be 200
print( "Request Code: {}".format( req.status_code ) )

# print output
if req.status_code == 200:
    print( "Found these buildings: " )
    response = req.json()
    print( response[ "results" ][ "bindings" ][ 0 ][ "building" ][ "value" ] )

Will connect to endpoint:  http://localhost:7200/repositories/Atlas-Robots
Request Code: 200
Found these buildings: 
http://linkedbuildingdata.net/ifc/resources20200408_172328/building_126


The above query finds available buildings in the database. We will instead load our triples into the selected repository.

In [76]:
# setup correct request headers
endpoint = "http://localhost:7200/repositories/SSoLDAC2024/statements"
headers = {
    "Content-Type": "application/x-turtle",
    "Accept": "application/json"
}

# Serialize the graph to Turtle format
rdf_data = g.serialize(format="turtle")

# send the data to the endpoint
response = requests.post(endpoint, data=rdf_data, headers=headers)

# Response status code, should be 200
print(f"Response status code: {response.status_code}")
print(f"Response content: {response.content.decode('utf-8')}")

Response status code: 204
Response content: 


By running this request, data gets loaded into the GraphDB installation. You can inspect the graph there and find the following:

<img src="figures/smallGraph.png" width="900" />